In [30]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
from nlpsig_networks.scripts.ffn_baseline_functions import (
    histories_baseline_hyperparameter_search
)

In [3]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [4]:
%run ../load_anno_mi.py

In [5]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-07-25 00:00:13
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-07-25 00:00:24
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-07-25 00:00:25
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-07-25 00:00:34
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-07-25 00:00:34


In [6]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)
    
sbert_embeddings.shape

(13551, 384)

In [7]:
client_index

[False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 False,
 T

In [8]:
client_transcript_id

tensor([  0,   0,   0,  ..., 131, 131, 131])

# Baseline: FFN using signatures

First, we dimension reduce these and then take signatures. We use the path signature as input to the FFN for classification.

We want to choose a dimension and signature depth such that the number of terms in the signature is _roughly_ 384 so that it is comparable to the number of features that we used for the previous baseline where we computed the mean of the history. Again, we are concatenating the features we obtain with the current utterance embedding.

In [9]:
num_epochs = 100
hidden_dim_sizes = [[128,128],[256,256],[512,512]]
dropout_rates = [0.5, 0.2, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

## Using log signature

In [10]:
log_signature_dimensions_and_sig_depths = [(28, 2), (10, 3), (6, 4)]

In [11]:
import signatory

[signatory.logsignature_channels(channels, depth)
 for (channels, depth) in log_signature_dimensions_and_sig_depths]

[406, 385, 406]

### Using UMAP

In [12]:
ffn_logsignature_umap_kfold, best_ffn_logsignature_umap_kfold, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    log_signature=True,
    dim_reduce_methods=["umap"],
    dimension_and_sig_depths=log_signature_dimensions_and_sig_depths,
    path_indices=client_index,
    split_ids=client_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_logsignature_umap_focal_{gamma}_kfold.csv",
    verbose=False
)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/ffn_logsignature_umap_focal_2_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/ffn_logsignature_umap_focal_2_kfold_best_model.csv


In [13]:
ffn_logsignature_umap_kfold.groupby(["dimension", "sig_depth", "hidden_dim", "dropout_rate", "learning_rate"]).mean()

/var/folders/l7/bnn1j6bs3bqfskq2jlq55yh80000gr/T/ipykernel_18434/3138393360.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ffn_logsignature_umap_kfold.groupby(["dimension", "sig_depth", "hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  \
dimension sig_depth hidden_dim dropout_rate learning_rate                       
6         4         (128, 128) 0.1          0.0001         0.507727  0.360960   
                                            0.0005         0.521780  0.353246   
                                            0.0010         0.533547  0.351780   
                               0.2          0.0001         0.510837  0.350623   
                                            0.0005         0.535985  0.351374   
...                                                             ...       ...   
28        2         (512, 512) 0.2          0.0005         0.558482  0.421841   
                                            0.0010         0.552446  0.418218   
                               0.5          0.0001         0.571651  0.424104   
                                            0.0005         0.569212  0.425719   
                                            0.0010         0.569182  0.416608   

                                                           precision  \
dimension sig_depth hidden_dim dropout_rate learning_rate              
6         4         (128, 128) 0.1          0.0001          0.361479   
                                            0.0005          0.356971   
                                            0.0010          0.356994   
                               0.2          0.0001          0.353179   
                                            0.0005          0.357155   
...                                                              ...   
28        2         (512, 512) 0.2          0.0005          0.424717   
                                            0.0010          0.419986   
                               0.5          0.0001          0.437790   
                                            0.0005          0.432750   
                                            0.0010          0.425096   

                                                             recall  \
dimension sig_depth hidden_dim dropout_rate learning_rate             
6         4         (128, 128) 0.1          0.0001         0.361194   
                                            0.0005         0.353992   
                                            0.0010         0.353643   
                               0.2          0.0001         0.351986   
                                            0.0005         0.353632   
...                                                             ...   
28        2         (512, 512) 0.2          0.0005         0.422468   
                                            0.0010         0.418650   
                               0.5          0.0001         0.418329   
                                            0.0005         0.422995   
                                            0.0010         0.414033   

                                                           valid_accuracy  \
dimension sig_depth hidden_dim dropout_rate learning_rate                   
6         4         (128, 128) 0.1          0.0001               0.674561   
                                            0.0005               0.677583   
                                            0.0010               0.671964   
                               0.2          0.0001               0.667085   
                                            0.0005               0.669950   
...                                                                   ...   
28        2         (512, 512) 0.2          0.0005               0.691214   
                                            0.0010               0.684947   
                               0.5          0.0001               0.700727   
                                            0.0005               0.698041   
                                            0.0010               0.685842   

                                                           valid_f1  \
dimension sig_depth hidden_dim dropout_rate learning_rate             


In [14]:
best_ffn_logsignature_umap_kfold

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,loss_function,gamma,k_fold,n_splits,batch_size,input_dim,dimension,sig_depth,method,log_signature
0,None,0.554031,0.419155,"[0.7003303445021235, 0.30224470439456214, 0.25...",0.423570,"[0.6856086856086856, 0.3087855297157623, 0.276...",0.416073,"[0.7156980950084398, 0.2959752321981424, 0.236...",None,0.710241,...,focal,2,True,5,64,790,28,2,umap,True
0,None,0.557994,0.420992,"[0.7042889390519187, 0.32575521644347555, 0.23...",0.424066,"[0.6941451990632318, 0.3276942355889724, 0.250...",0.418782,"[0.7147335423197492, 0.3238390092879257, 0.217...",None,0.707219,...,focal,2,True,5,64,790,28,2,umap,True
0,None,0.567139,0.437518,"[0.7096165191740413, 0.3223418573351279, 0.280...",0.438676,"[0.6947781885397413, 0.35298452468680913, 0.26...",0.438605,"[0.7251024837231734, 0.29659442724458207, 0.29...",None,0.709121,...,focal,2,True,5,64,790,28,2,umap,True
0,None,0.586344,0.448983,"[0.7226557949912639, 0.3574440480051898, 0.266...",0.456829,"[0.6989634970707526, 0.3753405994550409, 0.296...",0.443997,"[0.7480106100795756, 0.3411764705882353, 0.242...",None,0.706659,...,focal,2,True,5,64,790,28,2,umap,True
0,None,0.589392,0.433413,"[0.7356192194901872, 0.2915430267062315, 0.273...",0.444828,"[0.6910362364907819, 0.3635522664199815, 0.279...",0.432093,"[0.7863515794550278, 0.243343653250774, 0.2665...",None,0.706995,...,focal,2,True,5,64,790,28,2,umap,True


In [15]:
best_ffn_logsignature_umap_kfold["f1"].mean()

0.4320122761881547

In [16]:
best_ffn_logsignature_umap_kfold["precision"].mean()

0.4375938257198291

In [17]:
best_ffn_logsignature_umap_kfold["recall"].mean()

0.4299098254001956

In [18]:
np.stack(best_ffn_logsignature_umap_kfold["f1_scores"]).mean(axis=0)

array([0.71450216, 0.31986577, 0.26166889])

In [19]:
np.stack(best_ffn_logsignature_umap_kfold["precision_scores"]).mean(axis=0)

array([0.69290636, 0.34567143, 0.27420368])

In [20]:
np.stack(best_ffn_logsignature_umap_kfold["recall_scores"]).mean(axis=0)

array([0.73797926, 0.30018576, 0.25156446])

### Using random projections

In [21]:
ffn_logsignature_grp_kfold, best_ffn_logsignature_grp_kfold, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    log_signature=True,
    dim_reduce_methods=["gaussian_random_projection"],
    dimension_and_sig_depths=log_signature_dimensions_and_sig_depths,
    path_indices=client_index,
    split_ids=client_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_logsignature_grp_focal_{gamma}_kfold.csv",
    verbose=False
)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/ffn_logsignature_grp_focal_2_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/ffn_logsignature_grp_focal_2_kfold_best_model.csv


In [22]:
ffn_logsignature_grp_kfold.groupby(["dimension", "sig_depth", "hidden_dim", "dropout_rate", "learning_rate"]).mean()

/var/folders/l7/bnn1j6bs3bqfskq2jlq55yh80000gr/T/ipykernel_18434/2536377135.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ffn_logsignature_grp_kfold.groupby(["dimension", "sig_depth", "hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  \
dimension sig_depth hidden_dim dropout_rate learning_rate                       
6         4         (128, 128) 0.1          0.0001         0.596738  0.497316   
                                            0.0005         0.590520  0.492322   
                                            0.0010         0.583783  0.487006   
                               0.2          0.0001         0.599726  0.498823   
                                            0.0005         0.590642  0.494794   
...                                                             ...       ...   
28        2         (512, 512) 0.2          0.0005         0.602683  0.497789   
                                            0.0010         0.590764  0.492928   
                               0.5          0.0001         0.612498  0.496083   
                                            0.0005         0.600671  0.501886   
                                            0.0010         0.589727  0.495327   

                                                           precision  \
dimension sig_depth hidden_dim dropout_rate learning_rate              
6         4         (128, 128) 0.1          0.0001          0.492549   
                                            0.0005          0.487288   
                                            0.0010          0.482385   
                               0.2          0.0001          0.494740   
                                            0.0005          0.489605   
...                                                              ...   
28        2         (512, 512) 0.2          0.0005          0.495270   
                                            0.0010          0.487598   
                               0.5          0.0001          0.499469   
                                            0.0005          0.497436   
                                            0.0010          0.488539   

                                                             recall  \
dimension sig_depth hidden_dim dropout_rate learning_rate             
6         4         (128, 128) 0.1          0.0001         0.504843   
                                            0.0005         0.502508   
                                            0.0010         0.499384   
                               0.2          0.0001         0.505544   
                                            0.0005         0.507964   
...                                                             ...   
28        2         (512, 512) 0.2          0.0005         0.502661   
                                            0.0010         0.506012   
                               0.5          0.0001         0.493388   
                                            0.0005         0.508291   
                                            0.0010         0.508501   

                                                           valid_accuracy  \
dimension sig_depth hidden_dim dropout_rate learning_rate                   
6         4         (128, 128) 0.1          0.0001               0.726782   
                                            0.0005               0.722171   
                                            0.0010               0.720739   
                               0.2          0.0001               0.729200   
                                            0.0005               0.721500   
...                                                                   ...   
28        2         (512, 512) 0.2          0.0005               0.736363   
                                            0.0010               0.728774   
                               0.5          0.0001               0.742899   
                                            0.0005               0.740168   
                                            0.0010               0.735937   

                                                           valid_f1  \
dimension sig_depth hidden_dim dropout_rate learning_rate             


In [23]:
best_ffn_logsignature_grp_kfold

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,loss_function,gamma,k_fold,n_splits,batch_size,input_dim,dimension,sig_depth,method,log_signature
0,None,0.607682,0.487769,"[0.7375328083989501, 0.4107657801986543, 0.315...",0.488826,"[0.7298701298701299, 0.4256308100929615, 0.310...",0.487137,"[0.7453580901856764, 0.39690402476780184, 0.31...",None,0.745495,...,focal,2,True,5,64,790,28,2,gaussian_random_projection,True
0,None,0.613931,0.499005,"[0.7391977145577906, 0.4161073825503355, 0.341...",0.500963,"[0.729901269393512, 0.4299867899603699, 0.3430...",0.497419,"[0.7487340245960936, 0.40309597523219814, 0.34...",None,0.745831,...,focal,2,True,5,64,790,28,2,gaussian_random_projection,True
0,None,0.612102,0.503115,"[0.73437876960193, 0.4318957493018926, 0.34307...",0.502770,"[0.7347332850591359, 0.43283582089552236, 0.34...",0.503472,"[0.7340245960935616, 0.4309597523219814, 0.345...",None,0.743928,...,focal,2,True,5,64,790,28,2,gaussian_random_projection,True
0,None,0.609511,0.489517,"[0.7385348421679572, 0.4137709137709138, 0.316...",0.490876,"[0.7297551789077212, 0.4306764902880107, 0.312...",0.488690,"[0.7475283337352303, 0.39814241486068114, 0.32...",None,0.742026,...,focal,2,True,5,64,790,28,2,gaussian_random_projection,True
0,None,0.604481,0.493301,"[0.7306628172741488, 0.40918969264203664, 0.34...",0.493846,"[0.728996639462314, 0.41033623910336237, 0.342...",0.492770,"[0.732336628888353, 0.40804953560371515, 0.337...",None,0.744040,...,focal,2,True,5,64,790,28,2,gaussian_random_projection,True


In [24]:
best_ffn_logsignature_grp_kfold["f1"].mean()

0.49454129875840563

In [25]:
best_ffn_logsignature_grp_kfold["precision"].mean()

0.49545604731393744

In [26]:
best_ffn_logsignature_grp_kfold["recall"].mean()

0.4938975025158072

In [27]:
np.stack(best_ffn_logsignature_grp_kfold["f1_scores"]).mean(axis=0)

array([0.73606139, 0.4163459 , 0.3312166 ])

In [28]:
np.stack(best_ffn_logsignature_grp_kfold["precision_scores"]).mean(axis=0)

array([0.7306513 , 0.42589323, 0.32982361])

In [29]:
np.stack(best_ffn_logsignature_grp_kfold["recall_scores"]).mean(axis=0)

array([0.74159633, 0.40743034, 0.33266583])